In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import h5py
from os.path import isfile
from tqdm import tqdm
import random
from glob import glob
import skimage.io as io
import skimage.transform as tr
from ipywidgets import FloatProgress
from IPython.display import display
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import *
from keras.preprocessing.image import *
import SimpleITK as sitk
from utils import *

Using TensorFlow backend.


In [2]:
def load_data_age(files, name='age', size=None, crop=None):
    if isfile('data/x_' + name + '.npy'):
        print('Loading data from .npy files')
        x = np.load('data/x_' + name + '.npy')
        y = np.load('data/y_' + name + '.npy')
    else:
        print('Loading data from directory')
        files = glob(files)
        x, y = [], []
        for i in tqdm(range(len(files))):
            img = io.imread(files[i], plugin='simpleitk')
            if size and img.shape != size:
                img = tr.resize(img, size, mode='constant')
            if crop:
                img = img[crop[0]:-1*crop[0], crop[1]:-1*crop[1], crop[2]:-1*crop[2]]
            img = (img-img.min()) / (img.max()-img.min())
            x.append(img)
            file = files[i].split('_')
            age = int(file[3]) + int(file[4]) / 12.
            y.append(age)
        x = np.array(x)[..., np.newaxis].astype('float16')
        y = np.array(y)
        x, y = shuffle(x, y)
        np.save('data/x_' + name + '.npy', x)
        np.save('data/y_' + name + '.npy', y)
    return x, y

In [3]:
def n4_normalize(img):
    img = sitk.GetImageFromArray(img)
    img = sitk.Cast(img, sitk.sitkFloat32)
    mask = sitk.OtsuThreshold(img, 0, 1, 200)
    img = sitk.N4BiasFieldCorrection(img, mask)
    return sitk.GetArrayFromImage(img)

In [4]:
def to_classes(y, start, end, step=1):
    age_range = end - start
    num_classes = int(round(age_range / step))
    labels = np.zeros((len(y), num_classes))
    idx = (y - start) / step
    for i in range(len(idx)):
        labels[i, int(idx[i])] = 1
    return labels

In [5]:
x, y = load_data_age('data/*p*_Study/MHD/*.mhd', name='age_224', size=(36,368,368), crop=(6,72,72))
#x, y = load_data_age('data/*p*_Study/MHD/*.mhd', name='age_192', size=(36,320,320), crop=(6,64,64))

Loading data from .npy files


In [ ]:
y = to_classes(y, 14, 21)
y_.sum(axis=0)

In [6]:
y = lengthen(y, x.shape[1])
x = to_2d(x)

In [ ]:
y_min = y.min()
y_max = y.max()
y = (y-y_min) / (y_max-y_min)

In [ ]:
model_seg = UNet((x.shape[1:]), init_dim=32, depth=4, inc_rate=1, dropout=0.2)
model_seg.load_weights('weights/seg2d_9523_224_32_4_1_0.2.h5')
x_seg = model_seg.predict(x, batch_size=16, verbose=1)

In [ ]:
x_bone = x * x_seg
x_bone = x_bone.astype('float16')

In [ ]:
np.save('data/x_age_mask_224.npy', x_seg)

In [ ]:
np.save('data/x_age_bone_224.npy', x_bone)

In [7]:
x_bone = np.load('data/x_age_bone_224.npy')

In [8]:
x_bone.shape

(3480, 224, 224, 1)

In [ ]:
rndm = np.random.permutation(len(x))
for i in range(0,5):
    plt.figure(figsize=(13, 5))
    plt.subplot(1,3,1)
    plt.title('Raw Image')
    plt.imshow(x[rndm[i], ..., 0].astype('float32'))
    plt.subplot(1,3,2)
    plt.title('Mask')
    plt.imshow(x_seg[rndm[i], ..., 0].astype('float32'))
    plt.subplot(1,3,3)
    plt.title('Masked Image')
    plt.imshow(x_bone[rndm[i], ..., 0].astype('float32'))
    plt.show()

In [9]:
x = x_bone

In [6]:
print('Oldest candidate:', round(y.max(),2))
print('Youngest candidate:', round(y.min(),2))
print('Age range:', round(y.max()-y.min(),2))
print('Average age:', round(y.mean(),2))
print('Baseline:', round(abs(y.mean()-y).mean(),2))

Oldest candidate: 20.92
Youngest candidate: 14.42
Age range: 6.5
Average age: 17.44
Baseline: 1.16


The age among the candidates ranges from 14.5 to 21 years resulting in a range of 6.5 years. The average age is 17.5 and by always predicting this, one could get a mean absolute error (MAE) of 1.16 years. This is our baseline. The result we need to beat in order to have a network that learned a bias.

In [11]:
train_size = 2400

x_tr = x[:train_size]
y_tr = y[:train_size]

x_te = x[train_size:]
y_te = y[train_size:]

In [ ]:
tr_gen = ImageDataGenerator(rotation_range=0.1,
                         width_shift_range=0.1,
                         height_shift_range=0.1,
                         shear_range=0.1,
                         zoom_range=0.1,
                         horizontal_flip=True)
tr_gen.flow(x_tr, y_tr)

te_gen = ImageDataGenerator()
te_gen.flow(x_te, y_te)

In [12]:
i = Input(shape=x.shape[1:])
m = BatchNormalization()(i)
m = Conv2D(16, 3, activation='elu', padding='same')(m)
m = Conv2D(16, 3, strides=2, activation='elu', padding='same')(m)
m = Dropout(0.2)(m)

m = Conv2D(16, 3, activation='elu', padding='same')(m)
m = Conv2D(16, 3, strides=2, activation='elu', padding='same')(m)
m = Dropout(0.2)(m)

m = Conv2D(32, 3, activation='elu', padding='same')(m)
m = Conv2D(32, 3, strides=2, activation='elu', padding='same')(m)
m = Dropout(0.2)(m)

m = Conv2D(32, 3, activation='elu', padding='same')(m)
m = Conv2D(32, 3, strides=2, activation='elu', padding='same')(m)
m = Dropout(0.2)(m)

m = Conv2D(64, 3, activation='elu', padding='same')(m)
m = Conv2D(64, 3, strides=2, activation='elu', padding='same')(m)
m = Dropout(0.2)(m)

m = Conv2D(16, 1, activation='elu', padding='same')(m)
m = GlobalAveragePooling2D()(m)
o = Dense(1, activation='linear')(m)

model = Model(inputs=i, outputs=o)

In [13]:
model.load_weights('weights/lucky_shot/age2d_6203_192.h5')

In [ ]:
i = Input(shape=x_tr.shape[1:])

m = BatchNormalization()(i)
m = Conv2D(24, 3, activation='elu', padding='same')(m)
m = Conv2D(24, 3, activation='elu', padding='same')(m)
m = MaxPooling2D()(m)

m = BatchNormalization()(m)
m = Conv2D(32, 3, activation='elu', padding='same')(m)
m = Conv2D(32, 3, activation='elu', padding='same')(m)
m = MaxPooling2D()(m)

m = BatchNormalization()(m)
m = Conv2D(32, 3, activation='elu', padding='same')(m)
m = Conv2D(32, 3, activation='elu', padding='same')(m)
m = MaxPooling2D()(m)

m = BatchNormalization()(m)
m = Conv2D(40, 3, activation='elu', padding='same')(m)
m = Conv2D(40, 3, activation='elu', padding='same')(m)
m = MaxPooling2D()(m)

m = BatchNormalization()(m)
m = Conv2D(48, 3, activation='elu', padding='same')(m)
m = Conv2D(48, 3, activation='elu', padding='same')(m)
m = MaxPooling2D()(m)

m = BatchNormalization()(m)
m = Conv2D(56, 1, activation='elu', padding='same')(m)
m = GlobalAveragePooling2D()(m)
o = Dense(1, activation='linear')(m)

model = Model(inputs=i, outputs=o)

In [ ]:
i = Input(shape=x_tr.shape[1:])

m = BatchNormalization()(i)
m = Conv2D(32, 3, strides=2, activation='elu', padding='same')(m)
m = MaxPooling2D()(m)

m = BatchNormalization()(m)
m = Conv2D(16, 1, activation='elu', padding='same')(m)
n = Conv2D(32, 3, activation='elu', padding='same')(m)
m = Conv2D(32, 1, activation='elu', padding='same')(m)
m = Concatenate(axis=3)([m, n])
m = MaxPooling2D()(m)
m = Dropout(0.2)(m)

m = BatchNormalization()(m)
m = Conv2D(32, 1, activation='elu', padding='same')(m)
n = Conv2D(64, 3, activation='elu', padding='same')(m)
m = Conv2D(64, 1, activation='elu', padding='same')(m)
m = Concatenate(axis=3)([m, n])
m = MaxPooling2D()(m)
m = Dropout(0.2)(m)

m = BatchNormalization()(m)
m = Conv2D(64, 1, activation='elu', padding='same')(m)
n = Conv2D(128, 3, activation='elu', padding='same')(m)
m = Conv2D(128, 1, activation='elu', padding='same')(m)
m = Concatenate(axis=3)([m, n])
m = MaxPooling2D()(m)
m = Dropout(0.2)(m)

m = BatchNormalization()(m)
m = Conv2D(32, 1, activation='elu', padding='same')(m)
m = GlobalAveragePooling2D()(m)
o = Dense(1, activation='linear')(m)
#o = Dense(y.shape[1], activation='softmax')(m)

model = Model(inputs=i, outputs=o)

In [ ]:
#model.summary()

In [14]:
model.compile(loss='mse', optimizer=Adam(lr=0.001), metrics=['mae'])

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['acc'])

In [ ]:
save_best = ModelCheckpoint('weights/age2d_224_BNCCM32_clf.h5', save_best_only=True)

In [ ]:
stop_early = EarlyStopping(patience=20)

In [ ]:
model.fit(x_tr, y_tr, validation_data=(x_te, y_te), epochs=50, batch_size=4, callbacks=[save_best, stop_early])

In [ ]:
model = load_model('weights/lucky_shot/age2d_6203_192.h5')
model.compile(loss='mse', optimizer=Adam(lr=0.001), metrics=['mae'])

In [15]:
score = model.evaluate(x_te, y_te)
score

1080/1080 [==============================] - 30s     

[1.0794175175604996, 0.82619943442168065]

In [16]:
age_vec_tr = model.predict(x_tr, verbose=1)
age_vec_te = model.predict(x_te, verbose=1)

1080/1080 [==============================] - 29s     

In [17]:
age_vec_tr = np.reshape(age_vec_tr, (len(age_vec_tr) // 24, 24))
age_vec_te = np.reshape(age_vec_te, (len(age_vec_te) // 24, 24))

In [18]:
age_tr = shorten(y_tr, 24)
age_te = shorten(y_te, 24)

In [33]:
from sklearn.ensemble import ExtraTreesRegressor
trees = ExtraTreesRegressor(max_depth=3, criterion='mae')
trees.fit(age_vec_tr, age_tr)
pred_te = trees.predict(age_vec_te)
pred_tr = trees.predict(age_vec_tr)

In [34]:
score_te = abs(age_te - pred_te).mean()
score_tr = abs(age_tr - pred_tr).mean()
print(score_te, score_tr)

0.663888888889 0.530333333333


In [21]:
print(pred_te)

[ 15.40833333  18.9         15.99583333  17.47083333  18.375       19.04583333
  17.07916667  18.90833333  16.52916667  16.8875      18.46666667
  17.91666667  17.03333333  16.19166667  18.475       18.31666667
  19.32916667  18.34166667  18.77083333  18.34166667  15.59583333
  19.32916667  15.18333333  15.84166667  16.53333333  17.7         17.39166667
  15.3         17.95        17.03333333  18.52916667  17.74583333
  18.39583333  18.41666667  16.41666667  18.41666667  15.45416667
  19.32916667  16.25        17.17083333  17.47083333  16.33333333
  18.32083333  18.00416667  17.29166667]


In [22]:
print(age_te)

[ 16.75        18.25        15.75        15.91666667  17.75        17.83333333
  17.75        19.58333333  16.08333333  17.16666667  17.5         16.83333333
  16.25        15.58333333  18.25        17.83333333  18.16666667  18.25
  17.5         18.          15.83333333  17.          15.75        16.
  16.66666667  18.          16.83333333  15.          17.          17.25
  19.16666667  17.33333333  18.66666667  19.83333333  15.58333333
  17.08333333  14.66666667  20.08333333  16.08333333  16.41666667
  20.66666667  15.91666667  17.25        16.66666667  16.91666667]


In [24]:
from tpot import TPOTRegressor
tpot = TPOTRegressor(generations=10, population_size=50, scoring='neg_mean_absolute_error', verbosity=2)
tpot.fit(age_vec_tr, age_tr)

Optimization Progress:  17%|█▋        | 91/550 [00:20<01:31,  5.04pipeline/s]

Generation 1 - Current best internal CV score: 0.7194059508787926


Optimization Progress:  25%|██▍       | 136/550 [00:29<01:01,  6.73pipeline/s]

Generation 2 - Current best internal CV score: 0.7194059508787926


Optimization Progress:  33%|███▎      | 184/550 [00:53<01:12,  5.02pipeline/s]

...

Generation 8 - Current best internal CV score: 0.7058717254566849


Optimization Progress:  83%|████████▎ | 457/550 [02:38<00:51,  1.79pipeline/s]

Generation 9 - Current best internal CV score: 0.7058717254566849


Generation 10 - Current best internal CV score: 0.7058717254566849

Best pipeline: RandomForestRegressor(input_matrix, RandomForestRegressor__bootstrap=True, RandomForestRegressor__max_features=0.5, RandomForestRegressor__min_samples_leaf=10, RandomForestRegressor__min_samples_split=7, RandomForestRegressor__n_estimators=DEFAULT)


TPOTRegressor(config_dict={'sklearn.preprocessing.MinMaxScaler': {}, 'sklearn.linear_model.ElasticNetCV': {'tol': [1e-05, 0.0001, 0.001, 0.01, 0.1], 'l1_ratio': array([ 0.  ,  0.05,  0.1 ,  0.15,  0.2 ,  0.25,  0.3 ,  0.35,  0.4 ,
        0.45,  0.5 ,  0.55,  0.6 ,  0.65,  0.7 ,  0.75,  0.8 ,  0.85,
        0.9 ,..., 'squared_epsilon_insensitive']}, 'sklearn.linear_model.LassoLarsCV': {'normalize': [True, False]}},
       crossover_rate=0.1, cv=5, disable_update_check=False,
       generations=10, max_eval_time_mins=5, max_time_mins=None,
       mutation_rate=0.9, n_jobs=1, offspring_size=50, population_size=50,
       random_state=None, scoring=None, subsample=1.0, verbosity=2,
       warm_start=False)

In [35]:
tpot.score(age_vec_te, age_te)

0.67036217440406554

In [ ]:
tpot.export('trees.py')